# Power measurements quick view

This notebook provides a quick visualization of the data stored in `measurements/power_measurements.csv`. It parses the timestamps, cleans numeric columns, computes an estimated wattage, and plots the power trend alongside voltage and current for context.


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

# Load and normalize the data
DATA_PATH = Path("measurements") / "power_measurements.csv"
df = pd.read_csv(DATA_PATH)

# Parse timestamps and strip units
# Adjust the format string if your dates use a different ordering
# (e.g., day-first vs. month-first)
df = df.rename(columns={"datetime": "timestamp"})
df["timestamp"] = pd.to_datetime(df["timestamp"], format="%m/%d/%Y %H:%M", errors="coerce")

df["voltage_v"] = df["voltage"].str.rstrip("V").astype(float)
df["current_a"] = df["current"].str.rstrip("A").astype(float)
df["power_factor"] = pd.to_numeric(df["power_factor"], errors="coerce")

# Compute estimated real power in watts
# P = V * I * power_factor
df["power_w"] = df["voltage_v"] * df["current_a"] * df["power_factor"]

# Keep sorted, valid rows
clean_df = df.dropna(subset=["timestamp", "power_w"]).sort_values("timestamp").reset_index(drop=True)
clean_df.head()


In [ ]:
plt.style.use("seaborn-v0_8")
fig, ax = plt.subplots(figsize=(10, 6))

ax.plot(clean_df["timestamp"], clean_df["power_w"], label="Estimated power (W)", color="tab:red", linewidth=2)
ax.set_xlabel("Timestamp")
ax.set_ylabel("Power (W)", color="tab:red")
ax.tick_params(axis="y", labelcolor="tab:red")

# Add voltage/current on a secondary axis for quick comparison
ax2 = ax.twinx()
ax2.plot(clean_df["timestamp"], clean_df["voltage_v"], label="Voltage (V)", color="tab:blue", alpha=0.6)
ax2.plot(clean_df["timestamp"], clean_df["current_a"], label="Current (A)", color="tab:green", alpha=0.6)
ax2.set_ylabel("Voltage / Current")

# Combine legends
lines = ax.get_lines() + ax2.get_lines()
labels = [line.get_label() for line in lines]
ax2.legend(lines, labels, loc="upper left")

ax.set_title("Power measurements overview")
ax.grid(True, which="both", linestyle="--", alpha=0.5)
plt.xticks(rotation=30, ha="right")
plt.tight_layout()
plt.show()
